In [35]:
import pandas as pd
import numpy as np

import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score,\
recall_score, roc_auc_score, confusion_matrix, classification_report

warnings.filterwarnings(action='ignore')
plt.rc('font', family='Noto Sans CJK JP')

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.api import Logit

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler

In [37]:
df_all = pd.read_csv('./final_df_all_ver0.4.csv',engine = 'python')
df_all.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,...,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,...,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,...,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,200-299만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,...,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,...,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,...,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,300-399만원


In [38]:
df_all.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'overdue_count', 'income_ind_grp'],
      dtype='object')

In [39]:
score = df_all[['overdue_count','over_amt','over_period','loan_count',
                     'loan_amt_final2','loan_period',
                      'card_period',
                     'age','house_type','income_home_grp','expenditure_grp','flag_priv_biz_cd','dwell_type','job']]
score

,overdue_count,over_amt,over_period,loan_count,loan_amt_final2,loan_period,card_period,age,house_type,income_home_grp,expenditure_grp,flag_priv_biz_cd,dwell_type,job
0,1.0,5700000.0,2.0,9.0,70300000.0,79.0,99.0,30,아파트,900-999만원,100-199만원,N,자가,기능/노무직
1,0.0,0.0,0.0,2.0,3150000.0,6.0,126.0,66,아파트,200-299만원,99만원 이하,Y,자가,전업주부
2,0.0,0.0,0.0,5.0,391000000.0,79.0,50.0,43,아파트,500-599만원,100-199만원,N,자가,판매/서비스직
3,0.0,0.0,0.0,2.0,590000.0,5.0,25.0,28,아파트,500-599만원,99만원 이하,N,자가,무직/기타
4,0.0,0.0,0.0,4.0,367200000.0,40.0,75.0,52,아파트,500-599만원,200-299만원,N,자가,행정관리/전문직
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,0.0,0.0,0.0,2.0,7900000.0,17.0,0.0,35,아파트,500-599만원,99만원 이하,N,전세,사무직
1425,1.0,500000.0,6.0,1.0,1360000.0,25.0,0.0,42,아파트,300-399만원,100-199만원,N,전세,기능/노무직
1426,0.0,0.0,0.0,17.0,61340000.0,150.0,118.0,39,아파트,300-399만원,100-199만원,Y,자가,전업주부
1427,0.0,0.0,0.0,2.0,68000000.0,29.0,25.0,41,단독주택,300-399만원,100-199만원,N,자가,사무직


In [31]:
# 처리해야하는 명목형 변수
# house_type, income_home_grp, expenditure_grp, flag_priv_biz_cd, dwell_type, job

In [42]:
# 직업별
score['job'].replace(['기능/노무직','무직/기타','사무직','전업주부','판매/서비스직','학생(재수생포함)','행정관리/전문직'],
                    [18.08, 23.09, 16.51, 14.76, 17.42, 18.43, 16.09], inplace = True)

# 개인사업자 여부
score['flag_priv_biz_cd'].replace(['N','Y'],[18.55, 6.6], inplace = True)


# 주거형태별 
score['dwell_type'].replace(['사택/기타','영구임대','월세','자가','전세'],
                  [24.34, 0.71, 12.64, 17.68, 16.18], inplace = True)

# 가족 수입 별
score['income_home_grp'].replace(['99만원 이하','100-199만원','200-299만원','300-399만원','400-499만원',
                                 '500-599만원','600-699만원','700-799만원','800-899만원','900-999만원','1000만원 이상'],
                  [14.46, 20.19, 15.74, 16.48, 19.95, 15.07, 20.0, 9.26, 28.22, 11.24, 26.75], inplace = True)

# 사용금액별?
score['expenditure_grp'].replace(['99만원 이하','100-199만원','200-299만원','300-399만원','400-499만원',
                                 '500-599만원'],
                  [19.13,15.06,17.85,19.1, 21.92, 83.82], inplace = True)

# 주거 환경별
score['house_type'].replace(['기타','단독주택','상가주택','아파트','연립/다세대/빌라',
                                 '오피스텔'],
                  [3.76, 16.42, 37.66, 17.72, 16.57, 5.34], inplace = True)

In [43]:
score

,overdue_count,over_amt,over_period,loan_count,loan_amt_final2,loan_period,card_period,age,house_type,income_home_grp,expenditure_grp,flag_priv_biz_cd,dwell_type,job
0,1.0,5700000.0,2.0,9.0,70300000.0,79.0,99.0,30,17.72,11.24,15.06,18.55,17.68,18.08
1,0.0,0.0,0.0,2.0,3150000.0,6.0,126.0,66,17.72,15.74,19.13,6.60,17.68,14.76
2,0.0,0.0,0.0,5.0,391000000.0,79.0,50.0,43,17.72,15.07,15.06,18.55,17.68,17.42
3,0.0,0.0,0.0,2.0,590000.0,5.0,25.0,28,17.72,15.07,19.13,18.55,17.68,23.09
4,0.0,0.0,0.0,4.0,367200000.0,40.0,75.0,52,17.72,15.07,17.85,18.55,17.68,16.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,0.0,0.0,0.0,2.0,7900000.0,17.0,0.0,35,17.72,15.07,19.13,18.55,16.18,16.51
1425,1.0,500000.0,6.0,1.0,1360000.0,25.0,0.0,42,17.72,16.48,15.06,18.55,16.18,18.08
1426,0.0,0.0,0.0,17.0,61340000.0,150.0,118.0,39,17.72,16.48,15.06,6.60,17.68,14.76
1427,0.0,0.0,0.0,2.0,68000000.0,29.0,25.0,41,16.42,16.48,15.06,18.55,17.68,16.51


In [44]:
scaler = MinMaxScaler()
cols = score.columns

np_scaled = scaler.fit_transform(score)
df_scaled = pd.DataFrame(np_scaled, columns = cols)
df_scaled

,overdue_count,over_amt,over_period,loan_count,loan_amt_final2,loan_period,card_period,age,house_type,income_home_grp,expenditure_grp,flag_priv_biz_cd,dwell_type,job
0,0.090909,0.003794,0.013333,0.473684,0.039056,0.409326,0.562500,0.140625,0.411799,0.104430,0.000000,1.0,0.718155,0.398559
1,0.000000,0.000000,0.000000,0.105263,0.001750,0.031088,0.715909,0.703125,0.411799,0.341772,0.059191,0.0,0.718155,0.000000
2,0.000000,0.000000,0.000000,0.263158,0.217222,0.409326,0.284091,0.343750,0.411799,0.306435,0.000000,1.0,0.718155,0.319328
3,0.000000,0.000000,0.000000,0.105263,0.000328,0.025907,0.142045,0.109375,0.411799,0.306435,0.059191,1.0,0.718155,1.000000
4,0.000000,0.000000,0.000000,0.210526,0.204000,0.207254,0.426136,0.484375,0.411799,0.306435,0.040576,1.0,0.718155,0.159664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,0.000000,0.000000,0.000000,0.105263,0.004389,0.088083,0.000000,0.218750,0.411799,0.306435,0.059191,1.0,0.654676,0.210084
1425,0.090909,0.000333,0.040000,0.052632,0.000756,0.129534,0.000000,0.328125,0.411799,0.380802,0.000000,1.0,0.654676,0.398559
1426,0.000000,0.000000,0.000000,0.894737,0.034078,0.777202,0.670455,0.281250,0.411799,0.380802,0.000000,0.0,0.718155,0.000000
1427,0.000000,0.000000,0.000000,0.105263,0.037778,0.150259,0.142045,0.312500,0.373451,0.380802,0.000000,1.0,0.718155,0.210084


In [45]:
df_scaled['chaju_id'] = df_all['chaju_id']

In [46]:
df_scaled.columns

Index(['overdue_count', 'over_amt', 'over_period', 'loan_count',
       'loan_amt_final2', 'loan_period', 'card_period', 'age', 'house_type',
       'income_home_grp', 'expenditure_grp', 'flag_priv_biz_cd', 'dwell_type',
       'job', 'chaju_id'],
      dtype='object')

In [47]:
df_scaled = df_scaled[['chaju_id','overdue_count', 'over_amt', 'over_period', 'loan_count',
       'loan_amt_final2', 'loan_period', 'card_period', 'age', 'house_type',
       'income_home_grp', 'expenditure_grp', 'flag_priv_biz_cd', 'dwell_type',
       'job']]

In [48]:
df_scaled

,chaju_id,overdue_count,over_amt,over_period,loan_count,loan_amt_final2,loan_period,card_period,age,house_type,income_home_grp,expenditure_grp,flag_priv_biz_cd,dwell_type,job
0,C-1001,0.090909,0.003794,0.013333,0.473684,0.039056,0.409326,0.562500,0.140625,0.411799,0.104430,0.000000,1.0,0.718155,0.398559
1,C-1002,0.000000,0.000000,0.000000,0.105263,0.001750,0.031088,0.715909,0.703125,0.411799,0.341772,0.059191,0.0,0.718155,0.000000
2,C-1003,0.000000,0.000000,0.000000,0.263158,0.217222,0.409326,0.284091,0.343750,0.411799,0.306435,0.000000,1.0,0.718155,0.319328
3,C-1004,0.000000,0.000000,0.000000,0.105263,0.000328,0.025907,0.142045,0.109375,0.411799,0.306435,0.059191,1.0,0.718155,1.000000
4,C-1005,0.000000,0.000000,0.000000,0.210526,0.204000,0.207254,0.426136,0.484375,0.411799,0.306435,0.040576,1.0,0.718155,0.159664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,0.000000,0.000000,0.000000,0.105263,0.004389,0.088083,0.000000,0.218750,0.411799,0.306435,0.059191,1.0,0.654676,0.210084
1425,C-2994,0.090909,0.000333,0.040000,0.052632,0.000756,0.129534,0.000000,0.328125,0.411799,0.380802,0.000000,1.0,0.654676,0.398559
1426,C-2995,0.000000,0.000000,0.000000,0.894737,0.034078,0.777202,0.670455,0.281250,0.411799,0.380802,0.000000,0.0,0.718155,0.000000
1427,C-2996,0.000000,0.000000,0.000000,0.105263,0.037778,0.150259,0.142045,0.312500,0.373451,0.380802,0.000000,1.0,0.718155,0.210084


In [49]:
df_scaled.to_csv('./x_scaler_ver2.csv', index = False, float_format = '%.3f')